In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models

In [2]:
#load cifar10 dataset
(training_images, training_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
print(training_images.shape)

training_images = training_images.astype('float32')
test_images = test_images.astype('float')

170508288/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)


In [3]:
train_norm = training_images / 255.0 - 1
test_norm = test_images / 255.0 -1

In [4]:
#load base model (VGG16)
from keras.applications.vgg16 import VGG16

base_model = VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(32, 32, 3),
    pooling="max"
)
base_model.summary()



58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [5]:
#Freeze all layers in the base model 
base_model.trainable = False

#Create new model by adding two dense layers on top of base model
inputs = keras.Input((32, 32, 3))

x = base_model(inputs, training=False)

x = keras.layers.Dense(units=128, activation='relu')(x)

outputs = keras.layers.Dense(units=10, activation='softmax')(x)

model = keras.models.Model(inputs, outputs)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 14,781,642
Trainable params: 66,954
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss=keras.losses.SparseCategoricalCrossentropy(), 
              metrics=keras.metrics.SparseCategoricalAccuracy())

In [8]:
model.fit(x=train_norm, y=training_labels, epochs=10, validation_data=(test_norm, test_labels))

Epoch 1/10
1563/1563 [==============================] - 45s 23ms/step - loss: 1.3475 - sparse_categorical_accuracy: 0.5297 - val_loss: 1.2275 - val_sparse_categorical_accuracy: 0.5726
Epoch 2/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.1724 - sparse_categorical_accuracy: 0.5896 - val_loss: 1.2009 - val_sparse_categorical_accuracy: 0.5796
Epoch 3/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.1157 - sparse_categorical_accuracy: 0.6101 - val_loss: 1.1959 - val_sparse_categorical_accuracy: 0.5838
Epoch 4/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.0710 - sparse_categorical_accuracy: 0.6247 - val_loss: 1.1560 - val_sparse_categorical_accuracy: 0.5969
Epoch 5/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.0358 - sparse_categorical_accuracy: 0.6358 - val_loss: 1.1311 - val_sparse_categorical_accuracy: 0.6039
Epoch 6/10
1563/1563 [==============================] - 35s 22ms/step - loss: 1.

In [15]:
#Do a round of fine-tuning of the entire model

base_model.trainable = True

In [17]:

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), # Low learning rate
              loss=keras.losses.SparseCategoricalCrossentropy(), 
              metrics=keras.metrics.SparseCategoricalAccuracy())

In [ ]:
model.fit(x=train_norm, y=training_labels, epochs=10, validation_data=(test_norm, test_labels))

Epoch 1/10
1563/1563 [==============================] - 103s 64ms/step - loss: 0.7773 - sparse_categorical_accuracy: 0.7281 - val_loss: 0.7642 - val_sparse_categorical_accuracy: 0.7361
Epoch 2/10
1563/1563 [==============================] - 99s 63ms/step - loss: 0.5576 - sparse_categorical_accuracy: 0.8028 - val_loss: 0.7022 - val_sparse_categorical_accuracy: 0.7602
Epoch 3/10
1563/1563 [==============================] - 99s 63ms/step - loss: 0.4272 - sparse_categorical_accuracy: 0.8474 - val_loss: 0.6231 - val_sparse_categorical_accuracy: 0.7903
Epoch 4/10
1563/1563 [==============================] - 99s 63ms/step - loss: 0.3266 - sparse_categorical_accuracy: 0.8839 - val_loss: 0.6734 - val_sparse_categorical_accuracy: 0.7876
Epoch 5/10
1563/1563 [==============================] - 99s 63ms/step - loss: 0.2437 - sparse_categorical_accuracy: 0.9144 - val_loss: 0.6707 - val_sparse_categorical_accuracy: 0.7960
Epoch 6/10
1563/1563 [==============================] - 99s 64ms/step - loss: 0